In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Pistes de Recherche sur le Projet TAS

Etudes sur les données fournies par TAS. 
Pistes :
- Chargement des données
- Traitement pour l'affichage
- Traitement du bruit
- Etude autour de la segmentation des images
- Classification des images avec un CNN (Keras)
- Lien Spark et Keras


## 1.Charger les données

On charge les données stockées dans des fichiers .npy.

Les labels sont sous la forme :
``` 
    [0,0,0,1,0]
```
C'est donc un array de 5 cases :
- 1 : Urban Area
- 2 : Agricultural Territory
- 3 : Forest
- 4 : Wetlands
- 5 : Surface with Water


In [3]:
testLabel=np.load('INSA_data_images/test_labels_0_10_25.npy')
testRGB = np.load('INSA_data_images/test_RGB_0_10_25.npy')
trainLabel= np.load('INSA_data_images/train_labels_0_10_25.npy')
trainRGB = np.load('INSA_data_images/train_RGB_0_10_25.npy')


In [ ]:
print("shape of Test label ", testLabel.shape)
# 42805 images avec 5 labels possibles
print("shape of Test RGB ", testRGB.shape)
# 42805 images de 32*32
print("shape of Train Label ",trainLabel.shape)
print("shape of Train RGB ", trainRGB.shape)

## 2. Etude sur l'affichage des images

### 2.1 Prétraitement pixels

Si on essaie d'afficher les images brutes, elles apparaisse extrèmement sombre. On n'arrive pas a distinguer à l'oeil nu ce que dit le label.  
Un traitement est donc nécessaire sur les images pour pouvoir les afficher. 

Visiblement, les images ne sont ni codées en $[0..255]$ RGB, ni en % (entre 0 et 1) RGB. 
Nous avons donc fait des tests à la main pour essayer d'avoir une image lisible. 

2 méthodes ont été retenues :
- multiplier les pixels par 10
- Récupérer la valeur max de chaque image et diviser les pixels par cette valeur max. On a ainsi un affichage plus égalisé.

In [ ]:
 def myImage(line, data):
    image= np.zeros([32,32,3])
    for i in range(0,31):
        for j in range(0,31):
            image[i,j] = data[line,i,j]
    maxval = image.max()
    
    for i in range(0,31):
        for j in range(0,31):
            image[i,j] = image[i,j]/maxval
    return image

 def myImage2(line, data):
    image= np.zeros([32,32,3])
    for i in range(0,31):
        for j in range(0,31):            
            image[i,j] = data[line,i,j]*10
    return image

### 2.2 Traitement du bruit (images noires)

On se rend compte aussi que certaines images ont tous leur pixel à 0. Elles sont donc toute noires. Cependant elles sont quand même labélisées. 
On considère que c'est du bruit et nous décidons de filtrer le dataset pour suprimer ces images.

In [ ]:
def delete_black_image(data) :
    indextodelete = []
    for i in range(0, data.shape[0]):
        if np.count_nonzero(data[i])==0:
            indextodelete.append(i)
    return indextodelete
        

In [ ]:
indexDeleteTrain= delete_black_image(trainRGB)
indexDeleteTest= delete_black_image(testRGB)

In [ ]:
print("Nb d'images 100% noires dans Train RGB : ",len(indexDeleteTrain))

print("Nb d'images 100% noires dans Test RGB : ",len(indexDeleteTest))


In [ ]:
mask = np.ones(trainRGB.shape[0],dtype=bool) #np.ones_like(a,dtype=bool)
mask[indexDeleteTrain] = False
trainRGB_clean= trainRGB[mask]
trainlabel_clean= trainLabel[mask]
print("shape of cleaned Train RGB ", trainRGB_clean.shape)

mask = np.ones(testRGB.shape[0],dtype=bool) #np.ones_like(a,dtype=bool)
mask[indexDeleteTest] = False
testRGB_clean= testRGB[mask]
testlabel_clean= testLabel[mask]
print("shape of cleaned Train RGB ", testRGB_clean.shape)

In [ ]:
np.save('INSA_data_images/CLEAN_test_labels_0_10_25.npy', testlabel_clean )
np.save('INSA_data_images/CLEAN_test_RGB_0_10_25.npy', testRGB_clean)
np.save('INSA_data_images/CLEAN_train_labels_0_10_25.npy', trainlabel_clean)
np.save('INSA_data_images/CLEAN_train_RGB_0_10_25.npy', trainRGB_clean)

### 2.3 Affichage des images

D'abord des images de tests, puis des images d'entrainement. On affiche aussi les images noires (pour voir leur label).

In [ ]:
# Affichage des images de test
fig, axarr = plt.subplots(20, sharex=True,  figsize=(100,100))

for i in range(0,20):
    im=myImage(i,testRGB)
    axarr[i].imshow(im)

plt.show()


In [ ]:
# Affichage pour les images Black
fig, axarr = plt.subplots(10, sharex=True,  figsize=(20,20))


for i in range(0,10):
    im=myImage(indexDelete[i],trainRGB)
    axarr[i].imshow(im)
    if (trainLabel[indexDelete[i],0]==1.):
        axarr[i].set_title("Urban Area")
    if (trainLabel[indexDelete[i],1]==1.):
        axarr[i].set_title("Agricultural territory")
    if (trainLabel[indexDelete[i],2]==1.):
        axarr[i].set_title("Forest")
    if (trainLabel[indexDelete[i],3]==1.):
        axarr[i].set_title("wetlands")
    if (trainLabel[indexDelete[i],4]==1.):
        axarr[i].set_title("Surface with water")

plt.show()

In [ ]:
# Affichage des images de train avec leur label
fig, axarr = plt.subplots(40, sharex=True,  figsize=(300,300))


for i in range(0,40):
    im=myImage(i,trainRGB)
    axarr[i].imshow(im)
    if (trainLabel[i,0]==1.):
        axarr[i].set_title("Urban Area")
    if (trainLabel[i,1]==1.):
        axarr[i].set_title("Agricultural territory")
    if (trainLabel[i,2]==1.):
        axarr[i].set_title("Forest")
    if (trainLabel[i,3]==1.):
        axarr[i].set_title("wetlands")
    if (trainLabel[i,4]==1.):
        axarr[i].set_title("Surface with water")

plt.show()

## 3. Recherche pour la partie apprentissage 

### 3.1 Segmentation

Dispo avec Scikit Learn, méthode non supervisée de Felzenszwalb.

On a lu beaucoup de litérature sur la segmentation d'image mais nous n'avons en fait pas les données nécessaires pour pouvoir faire un entrainement et de la segmentation d'image car nos images d'entrainement n'ont pas de calques.    
Effectivement, nos images ont un label valabe pour la totalité de l'image et non pas seulement une partie de cette image.  

On obtient des contours (en jaune) sur l'image. Nos recherches n'ont pas été poursuivies plus loin, car nous avons recentré l'étude sur la classification des images et non pas la segmentation

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.segmentation import felzenszwalb
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

ModuleNotFoundError: No module named 'skimage'

In [ ]:
fig, ax = plt.subplots(20, sharex=True,  figsize=(100,100))

for i in range (0,20):
    img = myImage(i, trainRGB)
    segments_fz = felzenszwalb(img, scale=100, sigma=0.5, min_size=50)
    #print("Felzenszwalb number of segments: {}".format(len(np.unique(segments_fz))))
    ax[i].imshow(mark_boundaries(img, segments_fz))
    if (trainLabel[i,0]==1.):
        ax[i].set_title("Urban Area")
    if (trainLabel[i,1]==1.):
        ax[i].set_title("Agricultural territory")
    if (trainLabel[i,2]==1.):
        ax[i].set_title("Forest")
    if (trainLabel[i,3]==1.):
        ax[i].set_title("wetlands")
    if (trainLabel[i,4]==1.):
        ax[i].set_title("Surface with water")

plt.show()

### 3.2 Etude de CNN

D'après nos études comparatives de mméthodes d'apprentissage vues en TP, nous avons vu que les réseaux de neurones sont plus performants que KNN en terme de temps d'exécution. Une fois le réseau entrainé, la classification est quasi-instantanées.  
Les réseaux de neuronnes sont donc plus adaptés à notre scénario de cas d'utilisation pour ce projet. 

Afin d'obtenir un modèle de réseau de neurones "sur mesure" qui corresponde aux données que l'on possède, on a choisi de se concentrer sur les CNN, Convolutional Neural Network. 

Ci-après est présenté et décris un CNN fait à la main. Il n'est pas extrèmement performant mais permet de décrire et comprendre un cas d'étude complet.

Par la suite, on utilisera des CNN plus conséquants et dont l'architecture est reconnue pour des problèmes de classification (comme AlexNet pour les problèmes de Fetch-Decode). Ces méthodes seront étudiées dans un autre document en annexe.   

L'outils de prédilection pour aborder CNN est Keras, qui repose sur un Backend TensorFlow. Il a l'avantage d'être facile à comprendre pour débuter une étude. D'autre par, si les modèles se complexifie il est possible de continuer avec toutes les possibilités qu'offre Tensorflow.

WORKFLOW de KERAS : 
1. Training Data 
2. Create Model (keras.layers())
3. Configure model (model.compile())
4. Train model (model.fit())
5. On trained Model, inject testing data then A OR B
  A. Evaluate model (model.evaluate()) => loss
  B. get predictions (model.predict()) => prediction

 Pour la partie compile, il faut configurer :
 - Spécifier un Optimizer qui determine comment les poids sont mis à jour
 - Spécifier le type de la cost function or loss function
 - Spécifier la métrique a évaluer pendant le training et testing
 - Creer le graphe du model en utilisant le backend
 - ....
 

### 3.2 Modèle basique

Cette modèle est conçu manuellement (non basé sur une architecture concrète).

#### a. Présentation générale

Notre modèle est composé de 7 couches cachées (hidden layers). Grâce à sa simplicité, on peut décrire la fonctionnalité des couches.

L'avantage d'utiliser un CNN plutôt qu'un simple Multilayer Perceptron est qu'il nous permet de ne plus devoir centrer nos images sur l'objet recherché. Le CNN devient "translation-invariant", invariant selon les translations. 

D'autre part, le multilayered perceptron network est "fully connected", cela veut dire que tous les perceptrons d'une couche sont connectés à tous les perceptrons de la couche suivante, et ainsi de suite. Donc il devient très compliqué de faire un grand réseau profond. 
CNN permet de choisir comment relier les couches entre elles et ainsi diminuer le nombre de paramètres à définir.

#### b. CNN
CNN est un réseau de neurones dit "feedforward", nourri en avant.  

Pour commencer, on peut séparer les couches en 2 groupes :
- les couches 1 à 6 permettent d'extraire les features de l'image
- la couche 7 permet de classifier l'image étudiée.

Le 1er groupe est constitué de couches convolutionnelles, de courches denses et  de couches de max-pooling qui de trouver les features.

La couche 7, elle, est 100% connectée afin d'agir comme un classifier. Si sa fonction d'activation est "softmax", elle nous donne le label de l'image. Si c'est "sigmoid", elle nous donne la probabilité que l'image soit ce label.

<div class="row" style="margin-top: 10px">
    <div class="col-md-offset-3 col-md-6">
        <img src="img_notebook/cnn-schema1.jpg" style="margin-right: 0; width: 3500px;" />
    </div>
</div>

**Les courches convolutionnelle**
C'est elles qui permettent de détecter des critères spécifiques. Si elles trouvent ce qu'elles cherchent, elles ont une forte réponse.
C'est en fait un filtre convolutionnel qui se balade sur la totalité de l'image et qui fait une somme pondérée pour caque localisation.

Dans notre cas pour la 1ere couche convolutionnelle, on a une entrée de $32*32*3$. Notre couche doit obligatoirement être de profondeur 3, comme notre array en entrée. 
Ensuite pour choisir la largeur et la hauteur du filtre, il faut s'assurer que celui ci reste dans l'image (on ne peut pas avoir un filtre qui soit a moitié en dehors de l'image). Pour cela, on peut soit retirer des pixels sur les côtés, soit ajouter du pading à l'image.

La sortie de l'image passée au filtre convolutionnel de la couche est appelé **activation map**. Elle est de profondeur 1.

Exemple : si on a une image de $32*32*3$ et un filtre de taille $3*3*3$, l'activation map est de taille $30*30*1$. il y a 2 colones et 2 lignes de pixels coupés pour que le filtre puisse se balader uniquement dans l'image. L'activation map est donc une couche de $30*30=900$ neurones.

Il est aussi possible d'utiliser **plusieurs filtres**.  C'est le cas de notre 1ere couche convolutionnelle qui est de taille $30*30*32$. Et cest 32 filtres partages les même poids, donc on a $3*3*3*32 = 288$ poids.
En fait, pour chaque filtre, on déplace l'image d'un pixel. donc si on a 32 filtres, on a déplacé l'image 32 fois.

<div class="row" style="margin-top: 10px">
    <div class="col-md-offset-3 col-md-6">
        <img src="img_notebook/activation-maps-32-kernel.jpg" style="margin-right: 0; width: 3500px;" />
    </div>
</div>

**Hierarchiration des caractéristiques** (des features) :

CNN, a travers ses couches succesives permer de hierarchiser les features. On arrive a trouver des features par effet de cascade : On part d'une image, on observe une portion de cette image. Puis dans cette portion, on observe un eplus petite portion. Et c'est dans cette dernière que l'on trouve la feature. (C'est un peu Inception).

**Les couches de Max Pooling** :  
On a presque toujours une couche max pooling après une couche convolutionnelle.  
Le Pooling nous permet de réduire la taille de l'espace (uniquement la largeur et hauteur, pas la profondeur). On peut ainsi réduire le nombre de paramètres et éviter l'overfitting. (l'overfitting s'exprime lorsque notre modèle est "trop bon", il répond parfaitement aux données d'entrainement mais n'est pas robustes aux données de test).

Max pooling est la forme de pooling la plus utilisée : on applique un filtre de taille p et on récupère la valeur max de cette partie de l'image.

<div class="row" style="margin-top: 10px">
    <div class="col-md-offset-3 col-md-6">
        <img src="img_notebook/max-pooling-demo.jpg" style="margin-right: 0; width: 3500px;" />
    </div>
</div>

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

def createModel(inputshape,nClasses):
# empilement de Conv layers puis de Max pooling layers.
# Dropout permet d'éviter l'overfitting
# A la fin, on a une fully connected layer (Dense) suivie d'une sopftmax layer
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',activation='relu', input_shape=inputshape))
    # 32 filters/kernels with (3*3) size window.
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25)) # 0.25 : dropout ratio
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nClasses, activation='sigmoid'))
    #model.add(Dense(nClasses, activation='softmax'))
    
    return model

# 6 Conv layze, 1 fully-connected layer


In [16]:
inputshape =(32,32,3)
nclasses = 5
mod = createModel(inputshape, nclasses)
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
__________

#### c. Expérimentation

**Compiler le modèle**

In [17]:
import keras

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# traitement des données 
# DEPRECEATE (no need to trained model on prepared data)

for i in range (1, 43000):
    if i==100 or i==1000 or i==10000 or i==30000:
        print(i)
    trainRGB[i]=(myImage2(i, trainRGB))


In [9]:
# Pick random data :
#indices = np.random.randint(1000, size=1000)

# DONNEES NON TRAITEES
#data = trainRGB[indices]
#target = trainLabel[indices]
data = trainRGB_clean
target = trainlabel_clean
xtrain = data
ytrain = target
xtest = testRGB_clean
ytest = testLabel_clean



**Entrainer le modèle**  

Pour entrainer le modèle, il faut faire plusieurs choix :
- la taille de batch
- le nombre d'époche


In [110]:
history = mod.fit(xtrain,ytrain, batch_size=50, epochs=1, verbose=1,validation_data=(xtest, ytest))

Train on 163989 samples, validate on 42805 samples
Epoch 1/1
163989/163989 [==============================] - 1103s 7ms/step - loss: 0.7278 - acc: 0.7155 - val_loss: 0.7216 - val_acc: 0.7237


**Evaluation du modèle**  
C'est optionnel si on a passé les données de validation du modèle dans le fit comme ci-dessus.

In [111]:
mod.evaluate(xtest,ytest, verbose=1)

42805/42805 [==============================] - 93s 2ms/step


[0.7215780501387384, 0.7237238640429302]

**Prediction**  
On veut prédire le label de nos images de tests. 
Dans l'exemple ci-après, on prédit pour les images 101 à 110 du test RGB. Le modèle nous donne en pourcentage la classification de l'image pour tel ou tel label.

In [112]:
pred =mod.predict(testRGB_clean[20:30])

In [115]:
lab = testlabel_clean[20:30]
for i in range(0, len(pred)):
    print("prediction ", pred[i])
    print("label      ", lab[i])

prediction  [0.12229992 0.6103879  0.09480256 0.01460406 0.00769571]
label       [0. 1. 0. 0. 0.]
prediction  [0.2536406  0.40241262 0.35615438 0.24223669 0.14956376]
label       [0. 1. 0. 0. 0.]
prediction  [0.03598148 0.06905532 0.104638   0.20048258 0.2525326 ]
label       [0. 0. 0. 0. 1.]
prediction  [0.7078749  0.16889891 0.05113503 0.03536195 0.011864  ]
label       [1. 0. 0. 0. 0.]
prediction  [0.1905594  0.1298222  0.09139905 0.2150234  0.25065726]
label       [0. 0. 0. 1. 0.]
prediction  [0.22913529 0.53453684 0.09326366 0.01978889 0.00995544]
label       [0. 1. 0. 0. 0.]
prediction  [0.53851706 0.31449786 0.05257269 0.01578066 0.00531275]
label       [0. 1. 0. 0. 0.]
prediction  [0.02355546 0.07582994 0.20053986 0.21445827 0.15407501]
label       [0. 1. 0. 0. 0.]
prediction  [0.13049115 0.5958856  0.17758335 0.03681044 0.01986633]
label       [0. 0. 1. 0. 0.]
prediction  [0.6146936  0.18203664 0.04271176 0.02926459 0.0102383 ]
label       [0. 1. 0. 0. 0.]


#### d. Sauvegarder le modèle
Il peut être utile de réutiliser un modèle sans avoir à le réentrainer. Pour cela, il faut sauvegarder le modèle dans un fichier.  

**sauvegarder**

In [116]:
from keras.models import model_from_json 
model_json = mod.to_json()
with open("Keras_Model_Trained/modelTrained.json","w") as json_file:
    json_file.write(model_json)

mod.save_weights("Keras_Model_Trained/modelTrained.h5")

print("Saved model to disk")

Saved model to disk


**Recharger**

In [6]:
from keras.models import model_from_json 
jsonfile = open('Keras_Model_Trained/modelTrained.json', 'r')
loaded_model_json = jsonfile.read()
jsonfile.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("Keras_Model_Trained/modelTrained.h5")
print("Loaded model from disk")

Loaded model from disk


### 3.3 Modèle profonde

Ce modèle est basé sur une architecture existante (il s'agit de **DenseNet**), elle est plus profonde. Nous utilisons le technique appellé **Transfer Learning** pour adapter à notre cas d'utilisation.

#### a. DenseNet.
Bien que, plusieurs architectures sont proposés par plusieurs laboratoires / entreprises lors des challenges Open data (for example ImageNet). Dans le cas de problème, pour cause de contraint de temps de développement, nous étudions seulement les architectures "classique" déjà disponibles sur Keras: **Xception**, **VGG**, **ResNet**, etc. (Cette étude est détaillé dans rapport). 

Nous choisissons **DenseNet** pour le compromise entre précision et le temps d'apprentissage, plus précisément il s'agit le modèle **DenseNet121**. 

<div class="row" style="margin-top: 10px">
    <div class="col-md-12">
        <img src="img_notebook/DenseNet.png" style="margin-right: 0; width: 100%" />
    </div>
</div>



### 3.4 Résultats

#### a. Sur des données non traitées
**1er test :**
```
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 25s 6ms/step - loss: 1.3918 - acc: 0.3845 - val_loss: 1.3303 - val_acc: 0.3600
```
```
Train on 33600 samples, validate on 8400 samples
Epoch 1/1
33600/33600 [==============================] - 248s 7ms/step - loss: 0.9733 - acc: 0.6020 - val_loss: 0.7275 - val_acc: 0.6975
```
```
Train on 171000 samples, validate on 42805 samples
Epoch 1/1
170950/171000 [============================>.] - ETA: 0s - loss: 0.7488 - acc: 0.7100
```

On constate ici que plus on a de données à apprendre, meilleure est la classification.

**2eme tests sur les données filtrées :**
ici on a entrainé et testé le modèle après avoir enlevé toutes les images noires. On a une accuracy meilleure de 1,3 points par rapport au dernier résultat ci-dessus.
```
Train on 163989 samples, validate on 42805 samples
Epoch 1/1
163989/163989 [==============================] - 1103s 7ms/step - loss: 0.7278 - acc: 0.7155 - val_loss: 0.7216 - val_acc: 0.7237
```

#### b. Sur des données traitées
**myImage** :
Avec myImage, le traitement est fait en récupérant la valeur max des pixels et en divisant les autres pixels par cette valeur max. On égalise ainsi les couleurs de l'image pour l'oeil humain. 

Cependant, on se rend compte que le modèle ne sais pas du tout comment lire ces données. Le score est extrèmement faible, un peu moins de 9%.
```
Train on 33600 samples, validate on 8400 samples
Epoch 1/1
33600/33600 [==============================] - 342s 10ms/step - loss: nan - acc: 0.0882 - val_loss: nan - val_acc: 0.0888
```
**myImage2**:
Avec ce traitement myImage2, on fait simplement que multipuler par 10 tous les pixels de l'image. 
On retrouve ici le score atteint avec des données non traitées. 

Il n'est donc pas pertinent pour le modèle d'être entrainé sur des données pré-traitées. Le traitement reste pertinent pour l'affichage "humain".
```
Train on 33600 samples, validate on 8400 samples
Epoch 1/1
33600/33600 [==============================] - 256s 8ms/step - loss: 0.9152 - acc: 0.6212 - val_loss: 0.7036 - val_acc: 0.7150
```

### 3.5 Pistes à améliorer :

Nous avons fait une validation de notre modèle assez basique. Il faudrait faire du k-fold validation et si possible du n-fold validation. Ainsi on serait assurée de la robustesse du modèle.

D'autre part, il faudrait pousser les recherches sut l'architecture du réseau en lui même pour que le modèle puisse avoir un meilleur score. c'est une tâche très chronophage que nous avons choisi d'écarter du projet afin de pouvoir fournit un projet intégré dans sa totalité et non pas centralisé sur la partie traitement des données.

## 4. Partie avec Spark
L'objectif de ce projet étant d'avoir une architecture et un pipeline complet et pouvant être distribué, il faut aussi prévoir la distribution de la partie apprentissage et/ou prediction des données.

Pour cela, Spark est idéal car il nous permet de faire des calculs distribués sur d'autres machines et également faire du calcul sur GPU.   

Il existe des outils pour nous aider à combiner kéras et Spark. Nous avons choisi ***Elephas*** qui permet d'avoir pyspark et keras.   

Tout d'abord il faut lancer la configuration de spark pour Elephas. Puis on peut directement récupérer le modèle fait avec Keras. Il est également possible de donner à spark un modèle déjà pré-entrainé. Dans ce cas, il ne faut pas oublier de recompiler le modèle keras avant de le donner à Spark.  

Si l'on veut entrainer notre modèle sur spark, il faut mettre les données xtrain et ytrain dans un RDD. Elephas nous permet de préparer cette structure de données.  

Un modèle basique dans Elephase est un SparkModel. On l'initialise en lui passant une fréquence de mise à jour et un mode de parallelization dans un modèle keras compilé.
Après on fait juste le traditionnel *fit* sur les données RDD (même options que Keras, donc on peut lui donner un batch_size et un epochs)

In [7]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Elephas_App').setMaster('local[8]')
sc = SparkContext(conf=conf)

In [10]:
from elephas.utils.rdd_utils import to_simple_rdd
rdd = to_simple_rdd(sc, xtrain, ytrain)

Ici, on a un exemple d'entrainement de modele sur spark. Exécuté dans une petite VM, j'ai juste voulu testé que cela marchait. J'ai entrainé le modèle avce seulement 100 données. C'est pour cela que le score est seulement de  5%.

In [18]:
from elephas.spark_model import SparkModel

spark_model = SparkModel(mod, frequency='epoch', mode='synchronous')
spark_model.fit(rdd, batch_size=32, epochs=10, verbose=0,validation_split=0.1)

In [19]:
score =spark_model.master_network.evaluate(xtest, ytest, verbose=1)
print('accuracy ', score)

42805/42805 [==============================] - 74s 2ms/step
accuracy  [1.6097211157101752, 0.05018105361523187]


Si l'on veut donner à spark un modèle pré-entrainer, il faut charger le modèle (cf partie 3.4). En évaluant ce modèle, on le même taux de réussite que lorsque l'on ne passait pas par spark. Spark a pu récupéré le modèle pré-entrainé par Keras.

In [14]:
import keras

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
loaded_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])
sparkmod = SparkModel(loaded_model, frequency='epoch',mode='synchronous')
scoreT=sparkmod.master_network.evaluate(xtest, ytest, verbose=1)
print('accuracy ', scoreT)

42805/42805 [==============================] - 64s 1ms/step
accuracy  [0.7215780546252742, 0.7237238640429302]
